In [7]:
import json
import pandas as pd
import requests

In [8]:
path = "../../data/SQUAD_v2.0/train.json"
with open(path) as f:
    data = json.load(f)

In [9]:
def find_titles_by_text(query, limit = 10):
        url = "https://en.wikipedia.org/w/api.php"
        params = {
            "srsearch": query,
            "srlimit": limit,
            "srwhat" : "text",
            "action": "query",
            "format": "json",
            "list": "search",
        }
        res = requests.get(url, params)
        return res.json()

In [12]:
def build_page_dataset(data, max_questions = 50):
    res = pd.DataFrame(columns = ["question", "label"])
    for i, data_subset in enumerate(data["data"]):
        print(data_subset["title"])
        if i % 50 == 0: 
            print("Extracting subset: [{}]/[{}]".format(i, len(data["data"])))
        n_questions = 0
        for paragraph in data_subset["paragraphs"]:
            context = paragraph["context"]
            print(context)
            label = find_titles_by_text(context[:300], limit = 1)
            if len(label["query"]["search"]) > 0:
                label = label["query"]["search"][0]["title"]
            else:
                continue
            for question in paragraph["qas"]:
                if not question["is_impossible"]:
                    res.loc[len(res)] = {"question": question["question"], "label": label}
                    n_questions += 1
                if n_questions >= max_questions:
                    break
            if n_questions >= max_questions:
                break
    return res

In [13]:
page_dataset = build_page_dataset(data, max_questions = 1)

Beyoncé
Extracting subset: [0]/[442]
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Frédéric_Chopin
Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a 

KeyboardInterrupt: 